# Polars vs Pandas

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
# import os

# # Đường dẫn tới thư mục mới bạn muốn chuyển đến
# new_directory = r'D:\University\IDA\Polars'

# # Thay đổi thư mục làm việc hiện tại
# os.chdir(new_directory)

## Support Functions

In [3]:
def sort_dataframe(df, column):
    """
    Sắp xếp DataFrame theo thứ tự yêu cầu dựa trên giá trị của một cột.

    Parameters:
        - df: DataFrame chứa dữ liệu cần sắp xếp.
        - column: Tên của cột được sử dụng để sắp xếp DataFrame.

    Returns:
        DataFrame đã được sắp xếp theo thứ tự yêu cầu.
    """
    # Danh sách các tên file theo thứ tự yêu cầu
    file_order = [
        'numeric_dataset_100000.csv', 'numeric_dataset_200000.csv', 'numeric_dataset_500000.csv',
        'numeric_dataset_1000000.csv', 'numeric_dataset_2000000.csv', 'numeric_dataset_5000000.csv', 'numeric_dataset_10000000.csv',
        'object_dataset_100000.csv', 'object_dataset_200000.csv', 'object_dataset_500000.csv',
        'object_dataset_1000000.csv', 'object_dataset_2000000.csv', 'object_dataset_5000000.csv', 'object_dataset_10000000.csv',
        'mixed_dataset_100000.csv', 'mixed_dataset_200000.csv', 'mixed_dataset_500000.csv',
        'mixed_dataset_1000000.csv', 'mixed_dataset_2000000.csv', 'mixed_dataset_5000000.csv', 'mixed_dataset_10000000.csv'
    ]
    
    # Lấy danh sách các giá trị từ cột được chỉ định
    column_values = df[column].to_numpy().tolist()
    
    # Lọc và sắp xếp các giá trị theo thứ tự yêu cầu
    sorted_column_values = [value for value in file_order if value in column_values]
    
    # Ghi nhận các giá trị không có trong danh sách yêu cầu
    remaining_column_values = [value for value in column_values if value not in sorted_column_values]
    
    # Lọc ra các DataFrame theo thứ tự yêu cầu
    sorted_dfs = [df.filter(df[column] == value) for value in sorted_column_values]
    
    # Kết hợp các DataFrame đã lọc và sắp xếp với các DataFrame của các giá trị không có trong danh sách
    sorted_df = pl.concat(sorted_dfs)
    
    return sorted_df

In [4]:
def add_columns_from_filename(df, column_name):
    """
    Thêm các cột mới vào DataFrame dựa trên thông tin từ tên file.

    Parameters:
        - df: DataFrame chứa dữ liệu.
        - column_name: Tên của cột chứa thông tin từ tên file.

    Returns:
        DataFrame đã được mở rộng với các cột mới được tạo ra từ thông tin tên file.
    """
    df = df.with_columns([
        pl.col(column_name).str.extract(r'(numeric|object|mixed)', 1).alias('type'),
        pl.col(column_name).str.extract(r'_(\d+)\.csv$', 1).cast(pl.Int64).alias('size')
    ])
    return df

In [5]:
def triple_bar_plot_with_side_by_side_subplots(df, x, y_list, title):
    """
    Tạo biểu đồ cột ba với các subplot đặt cạnh nhau từ DataFrame df với danh sách các cột y đã chỉ định.
    
    Parameters:
        - df: DataFrame chứa dữ liệu.
        - x: Tên của cột được sử dụng trên trục x.
        - y_list: Danh sách các tên cột được sử dụng trên trục y.
        - title: Tiêu đề của biểu đồ.

    Returns:
        None. Hiển thị biểu đồ trực tiếp.
    """
    # Tạo danh sách các loại dữ liệu có tồn tại trong dataframe
    types = ['numeric', 'object', 'mixed', 'numeric_object']
    existing_types = [data_type for data_type in types if df.filter(pl.col('type') == data_type).height > 0]
    num_existing_types = len(existing_types)

    if num_existing_types == 0:
        print("No data to plot.")
        return

    # Tạo figure với số lượng subplot tương ứng với số loại dữ liệu có tồn tại
    fig = make_subplots(rows=1, cols=num_existing_types, shared_yaxes=True, subplot_titles=existing_types,
                        column_widths=[1/num_existing_types]*num_existing_types)

    # Thể hiện y đầu tiên là 'Pandas', y thứ hai là 'Polars' và y thứ ba là 'Polars (Lazy)'
    legend_names = ['Pandas', 'Polars', 'Polars (Lazy)']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Màu sắc cho các thư viện

    

    # Duyệt qua từng loại dữ liệu có tồn tại và tạo các subplot
    
    for col_idx, data_type in enumerate(existing_types):
        df_type = df.filter(pl.col('type') == data_type)
        
        traces = []
        for i, y in enumerate(y_list):
            trace = go.Bar(x=df_type[x].cast(pl.Utf8), 
                           y=df_type[y], 
                           name=legend_names[i],
                            marker_color=colors[i] , legendgroup= y_list[i], showlegend=(col_idx == 0))
            traces .append(trace)

        fig.add_traces(traces, rows= 1, cols=col_idx+ 1)
        fig.update_xaxes(title_text=x, row=1, col=col_idx + 1)

        # Hiển thị trục x cho mỗi subplot
    
    # Cập nhật layout
    fig.update_layout(title_text=title,
                      yaxis=dict(title='Time (s)'),
                      barmode='group',
                      legend=dict(title='Library', tracegroupgap=10),
                      height=600, width=1200)  # Điều chỉnh chiều rộng dựa trên số lượng subplot

    # Hiển thị biểu đồ
    fig.show()
    

## Read Time

In [6]:
# đọc file read_time bằng polars
read_time_df = pl.read_csv('read_time.csv')
read_time_df = sort_dataframe(read_time_df, 'filename')
read_time_df = add_columns_from_filename(read_time_df, 'filename')
print('read_time_df:\n', read_time_df)

read_time_df:
 shape: (21, 7)
┌─────┬────────────────────────┬──────────────┬──────────────┬────────────────┬─────────┬──────────┐
│     ┆ filename               ┆ pd_read_time ┆ pl_read_time ┆ lzfr_read_time ┆ type    ┆ size     │
│ --- ┆ ---                    ┆ ---          ┆ ---          ┆ ---            ┆ ---     ┆ ---      │
│ i64 ┆ str                    ┆ f64          ┆ f64          ┆ f64            ┆ str     ┆ i64      │
╞═════╪════════════════════════╪══════════════╪══════════════╪════════════════╪═════════╪══════════╡
│ 7   ┆ numeric_dataset_100000 ┆ 0.326724     ┆ 0.095151     ┆ 0.036753       ┆ numeric ┆ 100000   │
│     ┆ .csv                   ┆              ┆              ┆                ┆         ┆          │
│ 10  ┆ numeric_dataset_200000 ┆ 0.445535     ┆ 0.298182     ┆ 0.164238       ┆ numeric ┆ 200000   │
│     ┆ .csv                   ┆              ┆              ┆                ┆         ┆          │
│ 12  ┆ numeric_dataset_500000 ┆ 1.080389     ┆ 0.578336     

In [7]:
# Vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(read_time_df, 'size', ['pd_read_time', 'pl_read_time','lzfr_read_time'], 'Comparison of Pandas and Polars Read Time')

## Aggregation Time

In [8]:
# đọc file aggregation_time bằng polars
aggregation_time_df = pl.read_csv('aggregation_time.csv')
aggregation_time_df = sort_dataframe(aggregation_time_df, 'filename')
aggregation_time_df = add_columns_from_filename(aggregation_time_df, 'filename')
print("Aggregation_time_df: \n", aggregation_time_df)

Aggregation_time_df: 
 shape: (7, 19)
┌─────┬─────────────┬─────────────┬─────────────┬───┬────────────┬────────────┬─────────┬──────────┐
│     ┆ filename    ┆ pd_min_time ┆ pd_max_time ┆ … ┆ lzfr_media ┆ lzfr_std_t ┆ type    ┆ size     │
│ --- ┆ ---         ┆ ---         ┆ ---         ┆   ┆ n_time     ┆ ime        ┆ ---     ┆ ---      │
│ i64 ┆ str         ┆ f64         ┆ f64         ┆   ┆ ---        ┆ ---        ┆ str     ┆ i64      │
│     ┆             ┆             ┆             ┆   ┆ f64        ┆ f64        ┆         ┆          │
╞═════╪═════════════╪═════════════╪═════════════╪═══╪════════════╪════════════╪═════════╪══════════╡
│ 0   ┆ numeric_dat ┆ 0.055215    ┆ 0.008913    ┆ … ┆ 0.042398   ┆ 0.020991   ┆ numeric ┆ 100000   │
│     ┆ aset_100000 ┆             ┆             ┆   ┆            ┆            ┆         ┆          │
│     ┆ .csv        ┆             ┆             ┆   ┆            ┆            ┆         ┆          │
│ 3   ┆ numeric_dat ┆ 0.011484    ┆ 0.010153    ┆ … ┆

In [9]:
# Lọc ra các dòng có filename không phải là các file object
filtered_df = aggregation_time_df.filter(
    (pl.col('filename') != 'object_dataset_100000.csv') &
    (pl.col('filename') != 'object_dataset_200000.csv') &
    (pl.col('filename') != 'object_dataset_500000.csv') &
    (pl.col('filename') != 'object_dataset_1000000.csv') &
    (pl.col('filename') != 'object_dataset_2000000.csv') &
    (pl.col('filename') != 'object_dataset_5000000.csv') &
    (pl.col('filename') != 'object_dataset_10000000.csv')
)

# Vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(filtered_df, 'size', ['pd_min_time', 'pl_min_time'], 'Comparison of Pandas and Polars Min Time')
triple_bar_plot_with_side_by_side_subplots(filtered_df, 'size', ['pd_max_time', 'pl_max_time'], 'Comparison of Pandas and Polars Max Time')
triple_bar_plot_with_side_by_side_subplots(filtered_df, 'size', ['pd_mean_time', 'pl_mean_time'], 'Comparison of Pandas and Polars Mean Time')
triple_bar_plot_with_side_by_side_subplots(filtered_df, 'size', ['pd_median_time', 'pl_median_time'], 'Comparison of Pandas and Polars Median Time')
triple_bar_plot_with_side_by_side_subplots(filtered_df, 'size', ['pd_std_time', 'pl_std_time'], 'Comparison of Pandas and Polars STD Time')


## Query Time

In [10]:
# đọc query_time bằng polars
query_time_df = pl.read_csv('query_time.csv')
query_time_df = sort_dataframe(query_time_df, 'filename')
query_time_df = add_columns_from_filename(query_time_df, 'filename')
print("Query_time_df: \n",query_time_df)

Query_time_df: 
 shape: (7, 10)
┌─────┬─────────────┬─────────────┬─────────────┬───┬────────────┬────────────┬─────────┬──────────┐
│     ┆ filename    ┆ pd_query1_t ┆ pl_query1_t ┆ … ┆ pl_query2_ ┆ lzfr_query ┆ type    ┆ size     │
│ --- ┆ ---         ┆ ime         ┆ ime         ┆   ┆ time       ┆ 2_time     ┆ ---     ┆ ---      │
│ i64 ┆ str         ┆ ---         ┆ ---         ┆   ┆ ---        ┆ ---        ┆ str     ┆ i64      │
│     ┆             ┆ f64         ┆ f64         ┆   ┆ f64        ┆ f64        ┆         ┆          │
╞═════╪═════════════╪═════════════╪═════════════╪═══╪════════════╪════════════╪═════════╪══════════╡
│ 0   ┆ numeric_dat ┆ 0.01316     ┆ 0.059455    ┆ … ┆ 0.007122   ┆ 0.066558   ┆ numeric ┆ 100000   │
│     ┆ aset_100000 ┆             ┆             ┆   ┆            ┆            ┆         ┆          │
│     ┆ .csv        ┆             ┆             ┆   ┆            ┆            ┆         ┆          │
│ 3   ┆ numeric_dat ┆ 0.005986    ┆ 0.003104    ┆ … ┆ 0.003

In [11]:
# Vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(query_time_df, 'size', ['pd_query1_time', 'pl_query1_time'], 'Comparison of Pandas and Polars Query 1 Time')
triple_bar_plot_with_side_by_side_subplots(query_time_df, 'size', ['pd_query2_time', 'pl_query2_time'], 'Comparison of Pandas and Polars Query 2 Time')

## Sorting Time

In [12]:
# đọc sort_time bằng polars
sort_time_df = pl.read_csv('sort_time.csv')
sort_time_df = sort_dataframe(sort_time_df, 'filename')
sort_time_df = add_columns_from_filename(sort_time_df, 'filename')
print("Sort_time_df: \n",sort_time_df)

Sort_time_df: 
 shape: (14, 7)
┌─────┬────────────────────────┬──────────────┬──────────────┬────────────────┬─────────┬──────────┐
│     ┆ filename               ┆ pd_sort_time ┆ pl_sort_time ┆ lzfr_sort_time ┆ type    ┆ size     │
│ --- ┆ ---                    ┆ ---          ┆ ---          ┆ ---            ┆ ---     ┆ ---      │
│ i64 ┆ str                    ┆ f64          ┆ f64          ┆ f64            ┆ str     ┆ i64      │
╞═════╪════════════════════════╪══════════════╪══════════════╪════════════════╪═════════╪══════════╡
│ 0   ┆ numeric_dataset_100000 ┆ 0.107542     ┆ 0.029732     ┆ 0.057911       ┆ numeric ┆ 100000   │
│     ┆ .csv                   ┆              ┆              ┆                ┆         ┆          │
│ 3   ┆ numeric_dataset_200000 ┆ 0.201441     ┆ 0.036259     ┆ 0.101548       ┆ numeric ┆ 200000   │
│     ┆ .csv                   ┆              ┆              ┆                ┆         ┆          │
│ 5   ┆ numeric_dataset_500000 ┆ 0.611773     ┆ 0.120082    

In [13]:
# Vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(sort_time_df, 'size', ['pd_sort_time', 'pl_sort_time'], 'Comparison of Pandas and Polars Sort Time')

## Group by Time

In [14]:
# đọc grouping_time bằng polars
grouping_time_df = pl.read_csv('group_by.csv')
grouping_time_df = sort_dataframe(grouping_time_df, 'filename')
grouping_time_df = add_columns_from_filename(grouping_time_df, 'filename')
print("Grouping_time_df: \n",grouping_time_df)

Grouping_time_df: 
 shape: (21, 8)
┌─────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────┬──────────┐
│     ┆ filename     ┆ pd_cate_grou ┆ pl_cate_gro ┆ pd_str_grou ┆ pl_str_grou ┆ type    ┆ size     │
│ --- ┆ ---          ┆ p_by         ┆ up_by       ┆ p_by        ┆ p_by        ┆ ---     ┆ ---      │
│ i64 ┆ str          ┆ ---          ┆ ---         ┆ ---         ┆ ---         ┆ str     ┆ i64      │
│     ┆              ┆ f64          ┆ f64         ┆ f64         ┆ f64         ┆         ┆          │
╞═════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╪═════════╪══════════╡
│ 0   ┆ numeric_data ┆ 0.003069     ┆ 0.02612     ┆ 0.012159    ┆ 0.008639    ┆ numeric ┆ 100000   │
│     ┆ set_100000.c ┆              ┆             ┆             ┆             ┆         ┆          │
│     ┆ sv           ┆              ┆             ┆             ┆             ┆         ┆          │
│ 3   ┆ numeric_data ┆ 0.002658     ┆ 0.009598    ┆ 0.01

In [15]:
# Vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(grouping_time_df, 'size', ['pd_cate_group_by', 'pl_cate_group_by'], 'Comparison of Pandas and Polars Grouping Category Time')
triple_bar_plot_with_side_by_side_subplots(grouping_time_df, 'size', ['pd_str_group_by', 'pl_str_group_by'], 'Comparison of Pandas and Polars Grouping String Time')

## Join Time

In [16]:
# đọc join_time bằng polars
join_time_df = pl.read_csv('join_time.csv')
join_time_df = sort_dataframe(join_time_df, 'filename1')
join_time_df = sort_dataframe(join_time_df, 'filename2')
join_time_df = add_columns_from_filename(join_time_df, 'filename1')

# Xóa cột "type" hiện tại
join_time_df = join_time_df.drop("type")

# Tạo cột mới có giá trị "numeric_object"
join_time_df = join_time_df.with_columns(type=pl.lit("numeric_object"))

print("Join_time_df: \n",join_time_df)

Join_time_df: 
 shape: (7, 8)
┌─────┬─────────────┬─────────────┬─────────────┬─────────────┬────────────┬──────────┬────────────┐
│     ┆ filename1   ┆ filename2   ┆ pd_join_tim ┆ pl_join_tim ┆ lzfr_join_ ┆ size     ┆ type       │
│ --- ┆ ---         ┆ ---         ┆ e           ┆ e           ┆ time       ┆ ---      ┆ ---        │
│ i64 ┆ str         ┆ str         ┆ ---         ┆ ---         ┆ ---        ┆ i64      ┆ str        │
│     ┆             ┆             ┆ f64         ┆ f64         ┆ f64        ┆          ┆            │
╞═════╪═════════════╪═════════════╪═════════════╪═════════════╪════════════╪══════════╪════════════╡
│ 0   ┆ numeric_dat ┆ object_data ┆ 0.046244    ┆ 0.086153    ┆ 0.159308   ┆ 100000   ┆ numeric_ob │
│     ┆ aset_100000 ┆ set_100000. ┆             ┆             ┆            ┆          ┆ ject       │
│     ┆ .csv        ┆ csv         ┆             ┆             ┆            ┆          ┆            │
│ 3   ┆ numeric_dat ┆ object_data ┆ 0.734672    ┆ 0.044083   

In [17]:
# Vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(join_time_df, 'size', ['pd_join_time', 'pl_join_time'], 'Comparison of Pandas and Polars Join Time')

## Manipulation Time

In [18]:
# đọc manipulation_time bằng polars
manipulation_time_df = pl.read_csv('pd_vs_pl_manipulation_time.csv')
manipulation_time_df = sort_dataframe(manipulation_time_df, 'filename1')
manipulation_time_df = sort_dataframe(manipulation_time_df, 'filename2')
manipulation_time_df = add_columns_from_filename(manipulation_time_df, 'filename1')

# Xóa cột "type" hiện tại
manipulation_time_df = manipulation_time_df.drop("type")

# Tạo cột mới có giá trị "numeric_object"
manipulation_time_df = manipulation_time_df.with_columns(type=pl.lit("mixed"))

print("Manipulation_time_df: \n",manipulation_time_df)

Manipulation_time_df: 
 shape: (7, 8)
┌─────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬──────────┬───────┐
│     ┆ filename1    ┆ filename2    ┆ pd_manipulat ┆ pl_manipulat ┆ lzfr_manipu ┆ size     ┆ type  │
│ --- ┆ ---          ┆ ---          ┆ ion_time     ┆ ion_time     ┆ lation_time ┆ ---      ┆ ---   │
│ i64 ┆ str          ┆ str          ┆ ---          ┆ ---          ┆ ---         ┆ i64      ┆ str   │
│     ┆              ┆              ┆ f64          ┆ f64          ┆ f64         ┆          ┆       │
╞═════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╪══════════╪═══════╡
│ 0   ┆ object_datas ┆ numeric_data ┆ 0.260702     ┆ 0.133259     ┆ 0.040177    ┆ 100000   ┆ mixed │
│     ┆ et_100000.cs ┆ set_100000.c ┆              ┆              ┆             ┆          ┆       │
│     ┆ v            ┆ sv           ┆              ┆              ┆             ┆          ┆       │
│ 3   ┆ object_datas ┆ numeric_data ┆ 1.092597     ┆ 

In [19]:
# Vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(manipulation_time_df, 'size', ['pd_manipulation_time', 'pl_manipulation_time', 'lzfr_manipulation_time'], 'Comparison of Pandas and Polars Manipulation Time')

## Read and Manipulation

In [20]:
# đọc read_and_manipulation bằng polars
read_and_manipulation_time_df = pl.read_csv('read_and_manipulation.csv')
read_and_manipulation_time_df = sort_dataframe(read_and_manipulation_time_df, 'filename1')
read_and_manipulation_time_df = sort_dataframe(read_and_manipulation_time_df, 'filename2')
read_and_manipulation_time_df = add_columns_from_filename(read_and_manipulation_time_df, 'filename1')

# Xóa cột "type" hiện tại
read_and_manipulation_time_df = read_and_manipulation_time_df.drop("type")

# Tạo cột mới có giá trị "numeric_object"
read_and_manipulation_time_df = read_and_manipulation_time_df.with_columns(type=pl.lit("mixed"))

print("Read_and_manipulation_time_df: \n",read_and_manipulation_time_df)

Read_and_manipulation_time_df: 
 shape: (7, 8)
┌─────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬──────────┬───────┐
│     ┆ filename1    ┆ filename2    ┆ pd_read_mani ┆ pl_read_mani ┆ lzfr_read_m ┆ size     ┆ type  │
│ --- ┆ ---          ┆ ---          ┆ pulation_tim ┆ pulation_tim ┆ anipulation ┆ ---      ┆ ---   │
│ i64 ┆ str          ┆ str          ┆ e            ┆ e            ┆ _time       ┆ i64      ┆ str   │
│     ┆              ┆              ┆ ---          ┆ ---          ┆ ---         ┆          ┆       │
│     ┆              ┆              ┆ f64          ┆ f64          ┆ f64         ┆          ┆       │
╞═════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╪══════════╪═══════╡
│ 0   ┆ object_datas ┆ numeric_data ┆ 1.189066     ┆ 0.259445     ┆ 0.042706    ┆ 100000   ┆ mixed │
│     ┆ et_100000.cs ┆ set_100000.c ┆              ┆              ┆             ┆          ┆       │
│     ┆ v            ┆ sv           ┆       

In [21]:
# vẽ biểu đồ
triple_bar_plot_with_side_by_side_subplots(read_and_manipulation_time_df, 'size', ['pd_read_manipulation_time', 'pl_read_manipulation_time', 'lzfr_read_manipulation_time'], 'Comparison of Pandas and Polars Read and Manipulation Time')